In [ ]:
!nvidia-smi

Sun Jun 13 13:50:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 # Algoritmo

## Install

In [ ]:
pip install 'h5py<3.0.0'

     |████████████████████████████████| 2.9MB 6.9MB/s 
ERROR: tensorflow 2.5.0 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [ ]:
pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 44kB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 3.8MB 36.4MB/s 
     |████████████████████████████████| 512kB 41.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=4ab1440025c65a7df1f69127b87e25db1955999756b7592e8219b1d1fad0debf
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installati

In [ ]:
pip install keras==2.5.0rc0

     |████████████████████████████████| 1.2MB 5.3MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
!pip install wordninja

     |████████████████████████████████| 542kB 5.3MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-cp37-none-any.whl size=541553 sha256=453fc460d29c32744220e98f49b662b062c02f923e19785ec914f3bfeba450f5
  Stored in directory: /root/.cache/pip/wheels/22/46/06/9b6d10ed02c85e93c3bb33ac50e2d368b2586248f192a2e22a
Successfully built wordninja


## Imports

In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow.keras
import wordninja as wn

from sklearn import metrics
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense,Lambda,Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Parametri

In [ ]:
path = "/content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/"
elmo_path = "/content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/4/"

batch_size = 32
numEpochs = 10

start_fold = 1
end_fold = 11

nfolds = 10

nome_file = "Dataset_Completo.csv"

# Definizioni

In [ ]:
def arrayToSentence(x):
  string=''
  for a in x:
    string=string + a + ' '
  return string

def buildDataset():
  filecsv = open(path + nome_file, newline="")
  lettore = csv.reader(filecsv, delimiter=";")

  dataset_x = []
  dataset_y = []
  temp_y = []
  for a in lettore:
    dataset_y.append(a[0])
    if a[0] == 'dga':
      temp_y.append(1)
    else:
      temp_y.append(0)
    split = wn.split(a[3])
    sen = arrayToSentence(split)
    dataset_x.append(sen)

  filecsv.close()

  return dataset_x, dataset_y, temp_y

In [ ]:
def kfold(x, y, temp_y):
  # Divide the dataset into training + holdout and testing with folds
  sss = StratifiedKFold(n_splits=nfolds)

  fold = 0
  for train, test in sss.split(x, temp_y):
    print("Writing fold " + str(fold + 1) + " to csv...")
    fold += 1
    x_train, x_test, y_train, y_test, y_temp_train, y_temp_test = x[train], x[test], y[train], y[test], temp_y[train], temp_y[test]
    np.savetxt(path + "Dataset_2/x_train" + str(fold) + ".csv", x_train, fmt='%s', delimiter=';')
    np.savetxt(path + "Dataset_2/x_test" + str(fold) + ".csv", x_test, fmt='%s', delimiter=';')
    np.savetxt(path + "Dataset_2/y_train" + str(fold) + ".csv", y_train, fmt='%s', delimiter=';')
    np.savetxt(path + "Dataset_2/y_test" + str(fold) + ".csv", y_test, fmt='%s', delimiter=';')
    np.savetxt(path + "Dataset_2/temp_y_train" + str(fold) + ".csv", y_temp_train, fmt='%i', delimiter=';')
    np.savetxt(path + "Dataset_2/temp_y_test" + str(fold) + ".csv", y_temp_test, fmt='%i', delimiter=';')
  print("Files created")

In [ ]:
def encode(le, labels):
    enc = le.transform(labels)
    return tf.keras.utils.to_categorical(enc)  #era solo keras.utils...

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

# Preprocessing

In [ ]:
dataset_x, dataset_y, temp_y = buildDataset()
dataset_x = np.array(dataset_x)

le = preprocessing.LabelEncoder()
le.fit(dataset_y)

dataset_y_encode = encode(le, dataset_y)
dataset_y = np.array(dataset_y_encode)

temp_y = np.array(temp_y)

kfold(dataset_x, dataset_y, temp_y)

# Model

In [ ]:
fold = 9
# sss = StratifiedKFold(n_splits=nfolds)
# for train, test in sss.split(dataset_x, temp_y):
#   fold += 1
#   print('Fold: ', fold)
#   x_train, x_test, y_train, y_test, y_train_temp, y_test_temp = dataset_x[train], dataset_x[test], dataset_y[train], dataset_y[test], temp_y[train], temp_y[test]

# for fold in range(start_fold, end_fold):
print('Fold: ', fold, 'Epochs: ', numEpochs)
#Get fold by csv
x_train = np.genfromtxt(path + "Dataset_2/x_train" + str(fold) + ".csv", delimiter=';', dtype=None)
x_test = np.genfromtxt(path + "Dataset_2/x_test" + str(fold) + ".csv", delimiter=';', dtype=None)
y_train = np.genfromtxt(path + "Dataset_2/y_train" + str(fold) + ".csv", delimiter=';', dtype=None)
y_test = np.genfromtxt(path + "Dataset_2/y_test" + str(fold) + ".csv", delimiter=';', dtype=None)
y_train_temp = np.genfromtxt(path + "Dataset_2/temp_y_train" + str(fold) + ".csv", delimiter=';', dtype=None)
y_test_temp = np.genfromtxt(path + "Dataset_2/temp_y_test" + str(fold) + ".csv", delimiter=';', dtype=None)

print('Model Construction...')
model = None

#Parte costruzione del modello
# importo il modulo con la funzione di embedding ELMo
elmo = hub.Module(elmo_path)

# Definisco la funzione di embedding
def ELMoEmbedding(x):
  return elmo(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(128, activation='relu')(embedding)
pred = Dense(len(y_train[0]), activation='sigmoid')(dense)
model = Model(inputs=[input_text], outputs=pred)

model.compile('adam', 'binary_crossentropy', metrics=['accuracy',
    tf.keras.metrics.AUC(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall(),
    # tf.keras.metrics.Accuracy(), #aggiunto questo
    ])

print('...done!')

print('Train...')
#parte di training
Type = 'binary-'
with tf.compat.v1.Session() as session:
  earlystop = EarlyStopping(monitor='loss', patience=3)
  best_save = ModelCheckpoint(path + 'Saved/bestmodel' + str(fold) + '.hdf5', save_best_only=True, 
                              save_weights_only=False, 
                              monitor='val_loss', 
                              mode='min')
  tf.compat.v1.keras.backend.set_session(session)
  session.run(tf.compat.v1.global_variables_initializer())
  session.run(tf.compat.v1.tables_initializer())
  history = model.fit(x_train,y_train,
            batch_size=batch_size,
            epochs=numEpochs, 
            callbacks=[earlystop, best_save],
            validation_split=0.1
            )
  # model.save_weights('/content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/Saved_weights/' + Type + 'elmo-model' + str(fold) + '.h5')

  print('\nhistory dict:', history.history)

print('...done!')

print('Test...')
#Parte di test
with tf.compat.v1.Session() as session:
  tf.compat.v1.keras.backend.set_session(session)
  session.run(tf.compat.v1.global_variables_initializer())
  session.run(tf.compat.v1.tables_initializer())
  # model.load_weights('/content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/Saved_weights/'+ Type + 'elmo-model' + str(fold) + '.h5')
  best_model = load_model(path + 'Saved/bestmodel' + str(fold) + '.hdf5')
  predicts = best_model.predict(x_test, batch_size=batch_size)

# print(predicts)
y_preds = decode(le, predicts)
y_test_temp = np.where(y_test_temp == 1, 'dga', 'legit')
print('...done!')

print('Results:')
#Plotta i risultati
cm = metrics.confusion_matrix(y_test_temp, y_preds)
np.savetxt(path + 'Saved/confusion_matrix' + str(fold) + '.csv', cm, delimiter=',',  fmt='%i')
metrics1 = metrics.classification_report(y_test_temp, y_preds, output_dict=True, target_names=['legit', 'dga'])
print('Confusion_matrix:')
print(cm)
print('Classification_report:')
print(metrics1)
try:
    df1 = pd.read_csv(path + "Saved/metrics1.csv", index_col=[0])
    df1 = df1.append(pd.DataFrame(metrics1))
    df1.to_csv(path + "Saved/metrics1.csv")
except:
    pd.DataFrame(metrics1).to_csv(path + "Saved/metrics1.csv")

df_cm = pd.DataFrame(cm, index = [i for i in le.classes_],
                  columns = [i for i in le.classes_])
plt.figure(1, figsize = (10,7))
sn.heatmap(df_cm, annot=True, fmt="d")
plt.show()

Fold:  9 Epochs:  10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  if sys.path[0] == '':


Model Construction...
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
...done!
Train...
Train on 546668 samples, validate on 60741 samples
Epoch 1/10
546668/546668 [==============================] - 1800s 3ms/sample - loss: 0.1814 - acc: 0.9279 - auc: 0.9797 - precision: 0.9279 - recall: 0.9279 - val_loss: 0.1434 - val_acc: 0.9420 - val_auc: 0.9872 - val_precision: 0.9418 - val_recall: 0.9422
Epoch 2/10
546668/546668 [==============================] - 1782s 3ms/sample - loss: 0.1386 - acc: 0.9465 - auc: 0.9878 - precision: 0.9465 - recall: 0.9465 - val_loss: 0.1416 - val_acc: 0.9444 - val_auc: 0.9876 - val_precision: 0.9443 - val_recall: 0.9446
Epoch 3/10
546668/546668 [==============================] - 1789s 3ms/sample - loss: 0.1232 - acc: 0.9528 - auc: 0.9902 - precision

 # Test

In [ ]:
fold = 0
sss = StratifiedKFold(n_splits=nfolds)
for train, test in sss.split(dataset_x, temp_y):
  fold += 1
  print('Fold: ', fold)
  x_train, x_test, y_train, y_test, y_train_temp, y_test_temp = dataset_x[train], dataset_x[test], dataset_y[train], dataset_y[test], temp_y[train], temp_y[test]

# for fold in range(start_fold, end_fold):
#   print('Fold: ', fold, 'Epochs: ', numEpochs)

#   x_train = np.genfromtxt(path + "Dataset/x_train" + str(fold) + ".csv", delimiter=';', dtype=None)
#   x_test = np.genfromtxt(path + "Dataset/x_test" + str(fold) + ".csv", delimiter=';', dtype=None)
#   y_train = np.genfromtxt(path + "Dataset/y_train" + str(fold) + ".csv", delimiter=';', dtype=None)
#   y_test = np.genfromtxt(path + "Dataset/y_test" + str(fold) + ".csv", delimiter=';', dtype=None)
#   y_train_temp = np.genfromtxt(path + "Dataset/temp_y_train" + str(fold) + ".csv", delimiter=';', dtype=None)
#   y_test_temp = np.genfromtxt(path + "Dataset/temp_y_test" + str(fold) + ".csv", delimiter=';', dtype=None)

  #Parte costruzione del modello
  # importo il modulo con la funzione di embedding ELMo

  if fold != 1 and fold < 9:
    continue

  elmo = hub.Module(elmo_path)

  # Definisco la funzione di embedding
  def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

  # input_text = Input(shape=(1,), dtype=tf.string)
  # embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
  # dense = Dense(128, activation='relu')(embedding)
  # pred = Dense(len(y_train[0]), activation='sigmoid')(dense)
  # model = Model(inputs=[input_text], outputs=pred)

  # model.compile('adam', 'binary_crossentropy', metrics=['accuracy',
  #     tf.keras.metrics.AUC(),
  #     tf.keras.metrics.Precision(),
  #     tf.keras.metrics.Recall(),
  #     # tf.keras.metrics.Accuracy(), #aggiunto questo
  #     ])

  # print('...done!')

  # print('Train...')
  # #parte di training
  # Type = 'binary-'
  # with tf.compat.v1.Session() as session:
  #   earlystop = EarlyStopping(monitor='loss', patience=3)
  #   best_save = ModelCheckpoint(path + 'Saved/bestmodel' + str(fold) + '.hdf5', save_best_only=True, 
  #                               save_weights_only=False, 
  #                               monitor='val_loss', 
  #                               mode='min')
  #   tf.compat.v1.keras.backend.set_session(session)
  #   session.run(tf.compat.v1.global_variables_initializer())
  #   session.run(tf.compat.v1.tables_initializer())
  #   history = model.fit(x_train,y_train,
  #             batch_size=batch_size,
  #             epochs=numEpochs, 
  #             callbacks=[earlystop, best_save],
  #             validation_split=0.1
  #             )
  #   # model.save_weights('/content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/Saved_weights/' + Type + 'elmo-model' + str(fold) + '.h5')

  #   print('\nhistory dict:', history.history)

  # print('...done!')

  print(x_test)

  print('Test...')
  #Parte di test
  with tf.compat.v1.Session() as session:
    tf.compat.v1.keras.backend.set_session(session)
    session.run(tf.compat.v1.global_variables_initializer())
    session.run(tf.compat.v1.tables_initializer())
    # model.load_weights('/content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/Saved_weights/'+ Type + 'elmo-model' + str(fold) + '.h5')
    best_model = load_model(path + 'Saved/bestmodel' + str(fold) + '.hdf5')
    predicts = best_model.predict(x_test, batch_size=batch_size)

  # print(predicts)
  y_preds = decode(le, predicts)
  y_test_temp = np.where(y_test_temp == 1, 'dga', 'legit')
  print('...done!')

  print('Results:')
  #Plotta i risultati
  # cm = metrics.confusion_matrix(y_test_temp, y_preds)
  # np.savetxt(path + 'Saved/confusion_matrix' + str(fold) + '.csv', cm, delimiter=',',  fmt='%i')
  # metrics1 = metrics.classification_report(y_test_temp, y_preds, output_dict=True, target_names=['legit', 'dga'])
  # print('Confusion_matrix:')
  # print(cm)
  # print('Classification_report:')
  # print(metrics1)
  # try:
  #     df1 = pd.read_csv(path + "Saved/metrics1.csv", index_col=[0])
  #     df1 = df1.append(pd.DataFrame(metrics1))
  #     df1.to_csv(path + "Saved/metrics1.csv")
  # except:
  #     pd.DataFrame(metrics1).to_csv(path + "Saved/metrics1.csv")

  # df_cm = pd.DataFrame(cm, index = [i for i in le.classes_],
  #                   columns = [i for i in le.classes_])
  # plt.figure(1, figsize = (10,7))
  # sn.heatmap(df_cm, annot=True, fmt="d")
  # plt.show()

Fold:  1
['plastic bags s a com ' 'm zl track com ' 'miss slim ru ' ...
 'hh h hr x yy m ooo com ' 'mm jia j moj m uz mo com '
 'dv qom u aq yn com ']
Test...
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
...done!
Results:
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
['the peak magazine com sg ' 'furry tips com ' 'reuse co 2 com ' ...
 'go x as eh info ' 'od so ten woc a com ' 'zi ey qg haj my ']
Test...
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


InvalidArgumentError: ignored

In [ ]:
# fold = 0
# sss = StratifiedKFold(n_splits=nfolds)
# for train, test in sss.split(dataset_x, temp_y):
#   fold += 1
#   print('Fold: ', fold)
#   x_train, x_test, y_train, y_test, y_train_temp, y_test_temp = dataset_x[train], dataset_x[test], dataset_y[train], dataset_y[test], temp_y[train], temp_y[test]

start_fold, end_fold = 9, 10

for fold in range(start_fold, end_fold):
  print('Fold: ', fold, 'Epochs: ', numEpochs)
  #Get fold by csv
  x_train = np.genfromtxt(path + "Dataset/x_train" + str(fold) + ".csv", delimiter=';', dtype=None)
  x_test = np.genfromtxt(path + "Dataset/x_test" + str(fold) + ".csv", delimiter=';', dtype=None)
  y_train = np.genfromtxt(path + "Dataset/y_train" + str(fold) + ".csv", delimiter=';', dtype=None)
  y_test = np.genfromtxt(path + "Dataset/y_test" + str(fold) + ".csv", delimiter=';', dtype=None)
  y_train_temp = np.genfromtxt(path + "Dataset/temp_y_train" + str(fold) + ".csv", delimiter=';', dtype=None)
  y_test_temp = np.genfromtxt(path + "Dataset/temp_y_test" + str(fold) + ".csv", delimiter=';', dtype=None)

  if fold < 9:
    continue

  print('Model Construction...')
  model = None

  #Parte costruzione del modello
  # importo il modulo con la funzione di embedding ELMo
  elmo = hub.Module(elmo_path)

  # Definisco la funzione di embedding
  def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

  input_text = Input(shape=(1,), dtype=tf.string)
  embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
  dense = Dense(128, activation='relu')(embedding)
  pred = Dense(len(y_train[0]), activation='sigmoid')(dense)
  model = Model(inputs=[input_text], outputs=pred)

  model.compile('adam', 'binary_crossentropy', metrics=['accuracy',
      tf.keras.metrics.AUC(),
      tf.keras.metrics.Precision(),
      tf.keras.metrics.Recall(),
      # tf.keras.metrics.Accuracy(), #aggiunto questo
      ])

  print('...done!')

  # print('Train...')
  # #parte di training
  # Type = 'binary-'
  # with tf.compat.v1.Session() as session:
  #   earlystop = EarlyStopping(monitor='loss', patience=3)
  #   best_save = ModelCheckpoint(path + 'Saved/bestmodel' + str(fold) + '.hdf5', save_best_only=True, 
  #                               save_weights_only=False, 
  #                               monitor='val_loss', 
  #                               mode='min')
  #   tf.compat.v1.keras.backend.set_session(session)
  #   session.run(tf.compat.v1.global_variables_initializer())
  #   session.run(tf.compat.v1.tables_initializer())
  #   history = model.fit(x_train,y_train,
  #             batch_size=batch_size,
  #             epochs=numEpochs, 
  #             callbacks=[earlystop, best_save],
  #             validation_split=0.1
  #             )
  #   # model.save_weights('/content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/Saved_weights/' + Type + 'elmo-model' + str(fold) + '.h5')

  #   print('\nhistory dict:', history.history)

  # print('...done!')

  print('Test...', path + 'Saved/bestmodel_' + str(fold) + '_1.hdf5')
  #Parte di test
  with tf.compat.v1.Session() as session:
    tf.compat.v1.keras.backend.set_session(session)
    session.run(tf.compat.v1.global_variables_initializer())
    session.run(tf.compat.v1.tables_initializer())
    # model.load_weights('/content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/Saved_weights/'+ Type + 'elmo-model' + str(fold) + '.h5')
    best_model = load_model(path + 'Saved/bestmodel_' + str(fold) + '_1.hdf5')
    predicts = best_model.predict(x_test, batch_size=batch_size)

  # print(predicts)
  y_preds = decode(le, predicts)
  y_test_temp = np.where(y_test_temp == 1, 'dga', 'legit')
  print('...done!')

  # print('Results:')
  # #Plotta i risultati
  # cm = metrics.confusion_matrix(y_test_temp, y_preds)
  # np.savetxt(path + 'Saved/confusion_matrix' + str(fold) + '.csv', cm, delimiter=',',  fmt='%i')
  # metrics1 = metrics.classification_report(y_test_temp, y_preds, output_dict=True, target_names=['legit', 'dga'])
  # print('Confusion_matrix:')
  # print(cm)
  # print('Classification_report:')
  # print(metrics1)
  # try:
  #     df1 = pd.read_csv(path + "Saved/metrics1.csv", index_col=[0])
  #     df1 = df1.append(pd.DataFrame(metrics1))
  #     df1.to_csv(path + "Saved/metrics1.csv")
  # except:
  #     pd.DataFrame(metrics1).to_csv(path + "Saved/metrics1.csv")

  # df_cm = pd.DataFrame(cm, index = [i for i in le.classes_],
  #                   columns = [i for i in le.classes_])
  # plt.figure(1, figsize = (10,7))
  # sn.heatmap(df_cm, annot=True, fmt="d")
  # plt.show()

Fold:  9 Epochs:  10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  


Model Construction...
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
...done!
Test... /content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/Saved/bestmodel_9_1.hdf5
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


InvalidArgumentError: ignored

# SAVIO

In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow.keras
import wordninja as wn

from sklearn import metrics
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Lambda,Input

In [ ]:
nome_file_savio = "Dataset_Piccolo.csv"
def buildBinary():
  filecsv = open('/content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/' + nome_file_savio, newline="")
  lettore = csv.reader(filecsv, delimiter=";")

  sentencesTemp=[]
  labelsTemp=[]

  x_train=[]
  y_train=[]
  x_test=[]
  y_test=[]

  labelTemp=''
  count = 0

  dataset_x = []
  dataset_y = []
  for a in lettore:
    dataset_y.append(a[0])
    split = wn.split(a[3])
    sen = arrayToSentence(split)
    dataset_x.append(sen)
    if labelTemp=='':
      labelTemp=a[0]
      splitted = wn.split(a[3])
      sentence = arrayToSentence(splitted)
      sentencesTemp.append(sentence)
      labelsTemp.append(a[0])
    elif labelTemp==a[0]:
      splitted = wn.split(a[3])
      sentence = arrayToSentence(splitted)
      sentencesTemp.append(sentence)
      labelsTemp.append(a[0])
    elif a[0] != labelTemp:
      # print(sentencesTemp)
      # print(len(sentencesTemp))
      x_trainTemp, x_testTemp, y_trainTemp, y_testTemp = train_test_split(sentencesTemp,labelsTemp,test_size=0.6)
      x_train=x_train + x_trainTemp
      x_test=x_test + x_testTemp
      y_train=y_train + y_trainTemp
      y_test=y_test + y_testTemp
      sentencesTemp=[]
      labelsTemp=[]
      labelTemp=a[0]
      splitted = wn.split(a[3])
      sentence = arrayToSentence(splitted)
      sentencesTemp.append(sentence)
      labelsTemp.append(a[0])
      if count == 0:
        count = count + 1;
        # print(len(x_trainTemp))
        # print(x_train)
        # print(x_test)
        # print(y_train)
        # print(y_test)

  filecsv.close()
  # print(count)
  # print(dataset_x)
  # print(len(dataset_x))
  # print(dataset_y)
  # print(len(dataset_y))

  x_trainTemp, x_testTemp, y_trainTemp, y_testTemp = train_test_split(sentencesTemp,labelsTemp,test_size=0.6)
  x_train=x_train + x_trainTemp
  x_test=x_test + x_testTemp
  y_train=y_train + y_trainTemp
  y_test=y_test + y_testTemp

  x_train, y_train = shuffle(x_train, y_train)
  x_test, y_test = shuffle(x_test, y_test)


  return x_train, x_test, y_train, y_test

In [ ]:
batch_size = 32
numEpochs = 1 #10

print('Dataset Building...')

#x_train, x_test, y, y_test=DsB.build()
#Type = 'paper-matsnu-binary-'

x_train, x_test, y, y_test=buildBinary()
Type = 'binary-'

#x_train, x_test, y, y_test=DsBM.buildMulti()
#Type = 'multiclass-'


x_train=np.array(x_train)
x_test=np.array(x_test)


le = preprocessing.LabelEncoder()
le.fit(y)

# Definisco le funzioni per codificare e decodificare le labels
def encode(le, labels):
    enc = le.transform(labels)
    return tf.keras.utils.to_categorical(enc)  #era solo keras.utils...

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

y_enc = encode(le, y)

y_train=np.array(y_enc)

# print(y_enc)
# print(len(y_enc))
# print(y_train)
# print(len(y_train))

print(x_train)
print(y_train)

print('...done!')

print(len(y_enc[0]))

print('Model Construction...')
#Parte costruzione del modello
# importo il modulo con la funzione di embedding ELMo
elmo = hub.Module("/content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/4")

# Definisco la funzione di embedding
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]


input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense=Dense(128, activation='relu')(embedding)
pred=Dense(len(y_enc[0]), activation='sigmoid')(dense)
model=Model(inputs=[input_text], outputs=pred)


model.compile('adam', 'binary_crossentropy', metrics=['accuracy',
    tf.keras.metrics.AUC(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.Accuracy(), #aggiunto questo
    ])

print('...done!')

print('Train...')
#parte di training
with tf.compat.v1.Session() as session:
    tf.compat.v1.keras.backend.set_session(session)
    session.run(tf.compat.v1.global_variables_initializer())
    session.run(tf.compat.v1.tables_initializer())
    history = model.fit(x_train,y_train,
              batch_size=batch_size,
              epochs=numEpochs,
                #validation_data=[x_val, y_val]
              )
    model.save_weights('/content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/Saved_weights/' + Type + 'elmo-model.h5')

    print('\nhistory dict:', history.history)

print('...done!')

print('Test...')
print(x_test)
print(len(x_test))
#Parte di testing
with tf.compat.v1.Session() as session:
    tf.compat.v1.keras.backend.set_session(session)
    session.run(tf.compat.v1.global_variables_initializer())
    session.run(tf.compat.v1.tables_initializer())
    model.load_weights('/content/drive/MyDrive/Cyber Security/Elmo/prova_felipe/Saved_weights/'+ Type + 'elmo-model.h5')
    predicts = model.predict(x_test, batch_size=batch_size)

print(predicts)
y_preds = decode(le, predicts)

print(y_preds)
print(y_test)

print(len(predicts))
print(len(y_preds))

print('...done!')

print('Results:')
#Plotta i risultati
cm = metrics.confusion_matrix(y_test, y_preds)
print(metrics.classification_report(y_test, y_preds))

df_cm = pd.DataFrame(cm, index = [i for i in le.classes_],
                  columns = [i for i in le.classes_])
plt.figure(1, figsize = (10,7))
sn.heatmap(df_cm, annot=True, fmt="d")
plt.show()

#fpr_keras, tpr_keras, thresholds_keras = metrics.roc_curve(y_test.ravel(), predicts.ravel())
#auc_keras = auc(fpr_keras, tpr_keras)

#plotta i grafici
#plt.figure(2, figsize = (10,7))
#plt.plot([0, 1], [0, 1], 'k--')
#plt.plot(fpr_keras, tpr_keras, label='Binary (area = {:.3f})'.format(auc_keras))
#plt.xlabel('False positive rate')
#plt.ylabel('True positive rate')
#plt.title('ROC curve')
#plt.legend(loc='best')
#plt.show()